In [1]:
from scipy.io import loadmat, savemat
import numpy as np
import time
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics
from sklearn.decomposition import PCA
from sklearn.model_selection import cross_val_score
from sklearn import cross_validation

from keras.backend.tensorflow_backend import set_session
import tensorflow as tf
from keras import backend as K

from keras.layers import Input, Dense, Dropout
from keras.models import Model
from keras.models import Sequential
from keras.layers import Activation
from keras.wrappers.scikit_learn import KerasRegressor

from keras.optimizers import SGD, adam, nadam, Adagrad
from keras.regularizers import l1,l2

from keras.callbacks import EarlyStopping, CSVLogger
from keras.losses import mean_squared_logarithmic_error


import os
import os.path
import sys
import argparse
import time
import csv

D:\Users\Vishwesh\Anaconda3\envs\deep_l\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
Using TensorFlow backend.


In [2]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/cpu:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 8584910759355202278
, name: "/gpu:0"
device_type: "GPU"
memory_limit: 5073253171
locality {
  bus_id: 1
}
incarnation: 4362147803954039779
physical_device_desc: "device: 0, name: GeForce GTX 970M, pci bus id: 0000:01:00.0"
]


In [3]:
def acc_loss(y_true, y_pred):
    
    # Subtract mean value from expansions of true and pred
    x_true = y_true[:,1:66]
    x_pred = y_pred[:,1:66]
    
    # Normalize each vector
    comp_true = tf.conj(x_true)
    norm_true = x_true / tf.sqrt(tf.reduce_sum(tf.multiply(x_true,comp_true)))
    
    comp_pred = tf.conj(x_pred)
    norm_pred = x_pred / tf.sqrt(tf.reduce_sum(tf.multiply(x_pred,comp_pred)))
    
    comp_p2 = tf.conj(norm_pred)
    acc = tf.real(tf.reduce_sum(tf.multiply(norm_true,comp_p2)))
    acc = -1.0 * acc * acc
    
    loss_mse = K.mean(K.square(y_pred-y_true))
    
    return acc + loss_mse

In [4]:
def load_all(input_path,output_path,test_input_path,test_output_path):
    
    input = loadmat(input_path)
    output = loadmat(output_path)
    
    #X = np.array(input['train_input_shore'])
    X = np.array(input['train_shore_input'])
    y = np.array(output['train_shore_output'])
    
    # Get dimensions of arrays
    print('Training Data Information \n')
    x_size = X.shape
    print('Input Array Shape',x_size)
    y_size = y.shape
    print ('Output Array Shape',y_size)
    
    test_input = loadmat(test_input_path)
    test_output = loadmat(test_output_path)
    
    #X_test = np.array(test_input['test_input_shore'])
    X_test = np.array(test_input['test_shore_input'])
    y_test = np.array(test_output['test_shore_output'])
    
    # Get dimensions of Test data
    print('Testing Data information \n')
    x_size = X_test.shape
    print('Test Input Shape',x_size)
    y_size = y_test.shape
    print('Test Output Shape',y_size)
    
    return X,y,X_test,y_test

In [5]:
def build_nn2():
    model = Sequential()
    # Input layer with dimension 1 and hidden layer i with 128 neurons.
    model.add(Dense(50, input_shape=(50,)))
    model.add(Dense(400))
    model.add(Activation("relu"))
    #model.add(Dropout(0.6))
    # Hidden layer j with 64 neurons plus activation layer.
    model.add(Dense(200))
    model.add(Activation("relu"))
    #model.add(Dropout(0.5))
    # Hidden layer k with 64 neurons.
    model.add(Dense(66))
    model.add(Activation("relu"))
    #model.add(Dropout(0.5))
    model.add(Dense(200))
    model.add(Activation("relu"))
    #model.add(Dropout(0.5))
    model.add(Dense(400))
    #model.add(Activation("relu"))
    # Output Layer.
    model.add(Dense(50))
 
    # Model is derived and compiled using mean square error as loss
    # function, accuracy as metric and gradient descent optimizer.
    model.compile(loss='mse', optimizer='nadam', metrics=['mse','mae'])
    model.summary()
    return model

In [6]:
# Originally number of epochs was set to 1000, currently at 10.
def train_nn(model, X, y, out_dir, val_size=0.1, n_epoch=1000):
    if not os.path.exists(out_dir):
        os.makedirs(out_dir)
    csv_logger = CSVLogger(os.path.join(out_dir, 'results.csv'))

    model.fit(X, y, epochs=n_epoch, batch_size=10000, verbose=1, shuffle=True, validation_split=val_size, callbacks=[csv_logger])
    return model

In [7]:
def save_estimate(model, X, y, out_file, indices):
    #y_pred = model.predict(X)

    #y_pred = y_scaler.inverse_transform(y_pred)
    #y = y_scaler.inverse_transform(y)

    out_path = os.path.dirname(out_file)
    if not os.path.exists(out_path):
        os.makedirs(out_path)
     
    y_pred = model.predict(X)
    savemat(out_file, mdict={'out_pred': y_pred, 'out_true': y, 'indices': indices})

In [8]:
def save_test_set_prediction(model, out_file, X_test, y_test):
    
    # Get dimensions of arrays
    x_size = X_test.shape
    print('Hist 72: Input Array Shape',x_size)
    y_size = y_test.shape
    print ('Output Hist 72: Array Shape',y_size)
    
    # Make Predictions
    pred = model.predict(X_test)
    
    # If output path does not exist, create it
    out_path = os.path.dirname(out_file)
    if not os.path.exists(out_path):
        os.makedirs(out_path)
    
    savemat(out_file, mdict={'out_pred': pred, 'out_true': y_test})

In [9]:
def save_vishabyte_predictions(model, out_file):
    input_file_path = r'D:\Users\Vishwesh\PycharmProjects\shore_mapmri\Data\log_vish_feed_v3.mat'
    input_file_path = os.path.normpath(input_file_path)
    input_test = loadmat(input_file_path)
    X_f_t = np.array(input_test['log_vish_feed'])
    # Get dimensions of arrays
    x_size = X_f_t.shape
    print('Vishabyte: Input Array Shape',x_size)
    pred = model.predict(X_f_t)
    
    # If output path does not exist, create it
    out_path = os.path.dirname(out_file)
    if not os.path.exists(out_path):
        os.makedirs(out_path)
        
    # In vivo save matrix
    savemat(out_file, mdict={'out_pred': pred})

In [16]:
def main():

    #args = parse_args()
    work_dir = r'D:\Users\Vishwesh\PycharmProjects\shore_mapmri\dl_results'
    word_dir = os.path.normpath(work_dir)
    
    exp = 'Non_Neg_Shore_input_Shore_decayed_output_v4'
    itr = 20 # Estimated from CV

    train_input_data_path = r'D:\Users\Vishwesh\PycharmProjects\shore_mapmri\Data\log_train_input_shore_v5.mat'
    train_output_data_path = r'D:\Users\Vishwesh\PycharmProjects\shore_mapmri\Data\log_train_decayed_output_shore_v5.mat'
    train_input_data_path = os.path.normpath(train_input_data_path)
    train_output_data_path = os.path.normpath(train_output_data_path)
    
    test_input_data_path = r'D:\Users\Vishwesh\PycharmProjects\shore_mapmri\Data\log_test_input_shore_v5.mat'
    test_output_data_path = r'D:\Users\Vishwesh\PycharmProjects\shore_mapmri\Data\log_test_decayed_output_shore_v5.mat'
    test_input_data_path = os.path.normpath(test_input_data_path)
    test_output_data_path = os.path.normpath(test_output_data_path)
    
    print ("Loading data")
    X, y, X_b_test, y_b_test = load_all(train_input_data_path,train_output_data_path,test_input_data_path,test_output_data_path)
    indices = np.array(range(X.shape[0]))+1

    out_start_dir = os.path.join(work_dir,exp)
    if not os.path.exists(out_start_dir):
        os.makedirs(out_start_dir)

    seed1 = 46
    seed2 = 23
    
    kf = KFold(n_splits=5, random_state=seed1,shuffle=True,)
    
    fold_num = 0
    model_D = build_nn2()
    
    
    for train, test in kf.split(X):
        # Set up training / testing data
        fold_num += 1
        X_train = X[train,:]
        y_train = y[train,:]
        X_test = X[test,:]
        y_test = y[test,:]
        indices_train = indices[train]
        indices_test = indices[test]
        #X_train, X_test, y_train, y_test, X_scaler, y_scaler = norm_data(X_train, X_test, y_train, y_test)



        # Want to submit this w/ 1000 different initializations
        np.random.seed(seed=seed2)

        # Deep NN
        print ("Training DNN with %d iterations, fold %d" % (itr, fold_num))
        out_dir_DNN = os.path.join(out_start_dir, str(fold_num))
        model_D = train_nn(model_D, X_train, y_train, out_dir_DNN, n_epoch=itr, val_size=0.1)

        print ("Saving training outputs")
        end_dir = os.path.join(out_start_dir, str(fold_num), 'training.csv')
        save_estimate(model_D, X_train, y_train, end_dir,indices_train)

        print ("Saving testing outputs")
        end_dir = os.path.join(out_start_dir, str(fold_num), 'testing.csv')
        save_estimate(model_D, X_test, y_test, end_dir,indices_test)

    #print ("Saving Vishabyte outputs")
    #end_dir = os.path.join(out_start_dir, str('Vishabyte_Test'), 'result_vol_invivo_vish_b2000.mat')
    #save_vishabyte_predictions(model_D, end_dir)

    print ("Saving Histology blind test outputs")
    end_dir = os.path.join(out_start_dir, str('Hist_Blind_72_Test'), 'result.mat')
    save_test_set_prediction(model_D, end_dir, X_b_test, y_b_test)
    
    print ("Saving Vishabyte Predictions")
    end_dir = os.path.join(out_start_dir, str('Vishabyte_Preds'), 'result.mat')
    save_vishabyte_predictions(model_D, end_dir)

In [17]:
if __name__=='__main__':
    main()

Loading data
Training Data Information 

Input Array Shape (49995, 50)
Output Array Shape (49995, 50)
Testing Data information 

Test Input Shape (7272, 50)
Test Output Shape (7272, 50)
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_22 (Dense)             (None, 50)                2550      
_________________________________________________________________
dense_23 (Dense)             (None, 400)               20400     
_________________________________________________________________
activation_13 (Activation)   (None, 400)               0         
_________________________________________________________________
dense_24 (Dense)             (None, 200)               80200     
_________________________________________________________________
activation_14 (Activation)   (None, 200)               0         
_________________________________________________________________
dense_25 (Dense)      

35996/35996 [==============================] - 0s - loss: 16961066.7279 - mean_squared_error: 16961067.7284 - mean_absolute_error: 2227.5329 - val_loss: 10235868.0000 - val_mean_squared_error: 10235869.0000 - val_mean_absolute_error: 1665.5360
Epoch 5/20
35996/35996 [==============================] - 0s - loss: 17350073.8079 - mean_squared_error: 17350074.4194 - mean_absolute_error: 2245.7145 - val_loss: 10379023.0000 - val_mean_squared_error: 10379024.0000 - val_mean_absolute_error: 1717.0264
Epoch 6/20
35996/35996 [==============================] - 0s - loss: 15904219.6403 - mean_squared_error: 15904220.2518 - mean_absolute_error: 2214.4123 - val_loss: 10710292.0000 - val_mean_squared_error: 10710293.0000 - val_mean_absolute_error: 1735.7784
Epoch 7/20
35996/35996 [==============================] - 0s - loss: 14998342.7850 - mean_squared_error: 14998344.1187 - mean_absolute_error: 2163.1293 - val_loss: 10627659.0000 - val_mean_squared_error: 10627660.0000 - val_mean_absolute_error: 1

35996/35996 [==============================] - 0s - loss: 12286463.9934 - mean_squared_error: 12286465.3271 - mean_absolute_error: 1891.8015 - val_loss: 12501164.0000 - val_mean_squared_error: 12501166.0000 - val_mean_absolute_error: 1685.9946
Epoch 16/20
35996/35996 [==============================] - 0s - loss: 12539529.4028 - mean_squared_error: 12539529.6812 - mean_absolute_error: 1899.3986 - val_loss: 12374231.0000 - val_mean_squared_error: 12374233.0000 - val_mean_absolute_error: 1669.1255
Epoch 17/20
35996/35996 [==============================] - 0s - loss: 12515767.8432 - mean_squared_error: 12515768.5660 - mean_absolute_error: 1890.3424 - val_loss: 12708633.0000 - val_mean_squared_error: 12708633.0000 - val_mean_absolute_error: 1696.3859
Epoch 18/20
35996/35996 [==============================] - 0s - loss: 12651030.0326 - mean_squared_error: 12651031.0885 - mean_absolute_error: 1910.8975 - val_loss: 11588904.0000 - val_mean_squared_error: 11588906.0000 - val_mean_absolute_error

35996/35996 [==============================] - 0s - loss: 10406047.1479 - mean_squared_error: 10406047.3704 - mean_absolute_error: 1750.6974 - val_loss: 10193221.0000 - val_mean_squared_error: 10193221.0000 - val_mean_absolute_error: 1556.5679
Epoch 7/20
35996/35996 [==============================] - 0s - loss: 10608130.2658 - mean_squared_error: 10608130.8773 - mean_absolute_error: 1772.0069 - val_loss: 10010754.0000 - val_mean_squared_error: 10010756.0000 - val_mean_absolute_error: 1534.7163
Epoch 8/20
35996/35996 [==============================] - 0s - loss: 10270088.5344 - mean_squared_error: 10270089.0347 - mean_absolute_error: 1743.2407 - val_loss: 9559535.0000 - val_mean_squared_error: 9559535.0000 - val_mean_absolute_error: 1498.7858
Epoch 9/20
35996/35996 [==============================] - 0s - loss: 9953170.2142 - mean_squared_error: 9953171.2148 - mean_absolute_error: 1713.0647 - val_loss: 9572194.0000 - val_mean_squared_error: 9572195.0000 - val_mean_absolute_error: 1498.66